<a href="https://colab.research.google.com/github/adamggibbs/marine-carbonate-system-ml-prediction/blob/master/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# IMPORTS

import os
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split

# data dirs
data_dir = '/content/drive/MyDrive/Adam Gibbs/data/'
training_dir=data_dir + 'training/'
# ship_dir = data_dir + 'ship/'
# glider_dir = data_dir + 'glider/'
# wcoa_dir = data_dir + 'wcoa/'

In [ ]:
# load in txt data

ship_input = np.loadtxt(data_dir+'ship_tpso_input.csv', 
                        delimiter=",", 
                        dtype='str')
ship_output = np.loadtxt(data_dir+'ship_ph_output.csv', 
                        delimiter=",", 
                        dtype='str')

glider_input = np.loadtxt(data_dir+'glider_tpso_input.csv', 
                        delimiter=",", 
                        dtype='str')
glider_output = np.loadtxt(data_dir+'glider_ph_output.csv', 
                        delimiter=",", 
                        dtype='str')

wcoa_input = np.loadtxt(data_dir+'wcoa_tpso_input.csv', 
                        delimiter=",", 
                        dtype='str')
wcoa_output = np.loadtxt(data_dir+'wcoa_ph_output.csv', 
                        delimiter=",", 
                        dtype='str')

display(ship_input)
display(ship_output)

display(glider_input)
display(glider_output)

display(wcoa_input)
display(wcoa_output)


array([['20190511', '36.798', '-121.8462', ..., '8.6075', '33.9971',
        '94.2'],
       ['20190511', '36.798', '-121.8462', ..., '8.581', '34.001',
        '93.3'],
       ['20190511', '36.798', '-121.8462', ..., '9.1717', '33.9202',
        '112.5'],
       ...,
       ['20210514', '33.8167', '-121.84299999999999', ..., '10.4428',
        '33.5476', '185.3340941'],
       ['20210514', '33.8167', '-121.84299999999999', ..., '14.2987',
        '32.9877', '278.9397923'],
       ['20210514', '33.8167', '-121.84299999999999', ..., '14.5308',
        '32.994', '278.0892658']], dtype='<U19')

array(['7.730970739061131', '7.727680727079802', '7.762957925077376',
       '7.796313545797834', '7.894515063962825', '7.922467194824854',
       '7.949019302145861', '8.110336252729159', '8.147358201081984',
       '8.161302276567161', '7.726647094575305', '7.724865826450486',
       '7.779525569512648', '7.8188347213886535', '7.975602620866045',
       '8.112301324153226', '8.166555042139361', '7.639386777723472',
       '7.620494492693099', '7.621298192884223', '7.612505782805837',
       '7.662056525344349', '7.651942051544728', '7.623953382652679',
       '7.614173175119525', '7.6210066186145', '7.662241823366274',
       '7.758789892210932', '8.015863094803015', '7.705792882039213',
       '7.852819188090235', '7.905419022424352', '7.975348696746949',
       '8.124847096791852', '8.126777622903171', '8.132326437978204',
       '7.73290908989093', '7.716880201138217', '7.697879025575348',
       '7.677102210336942', '7.651149192505201', '7.607690952127642',
       '7.596676716927

array([['20190508', '36.736', '-122.011', ..., '6.017',
        '34.233000000000004', '19.45'],
       ['20190508', '36.736', '-122.011', ..., '6.026', '34.232',
        '19.42'],
       ['20190508', '36.736', '-122.011', ..., '6.035', '34.231',
        '19.42'],
       ...,
       ['20190905', '36.796', '-121.838', ..., '13.873', '33.63',
        '247.89'],
       ['20190905', '36.796', '-121.838', ..., '14.398', '33.611',
        '266.55'],
       ['20190905', '36.796', '-121.838', ..., '14.405', '33.622',
        '281.73']], dtype='<U18')

array(['7.5485', '7.5486', '7.5484', ..., '8.0313', '8.0642', '8.1087'],
      dtype='<U18')

array([['20160518', '36.81', '-121.818', '15.0', '13.34', '33.143',
        '272.8'],
       ['20160518', '36.81', '-121.818', '5.1', '13.96',
        '33.138000000000005', '273.9'],
       ['20160518', '36.81', '-121.818', '3.4', '14.0', '33.139',
        '274.0'],
       ['20160518', '36.795', '-121.848', '251.2', '7.89', '34.05',
        '67.8'],
       ['20160518', '36.795', '-121.848', '151.0', '8.62', '33.954',
        '92.6'],
       ['20160518', '36.795', '-121.848', '100.5', '8.87', '33.908',
        '104.4'],
       ['20160518', '36.795', '-121.848', '40.8', '10.09', '33.617',
        '165.0'],
       ['20160518', '36.795', '-121.848', '30.4', '10.17', '33.548',
        '177.7'],
       ['20160518', '36.795', '-121.848', '20.0', '10.53', '33.399',
        '190.5'],
       ['20160518', '36.795', '-121.848', '10.5', '12.65', '33.321',
        '238.8'],
       ['20160518', '36.784', '-121.87200000000001', '100.2', '8.67',
        '33.959', '91.2'],
       ['20160518', '36.784', 

array(['8.034862400919026', '8.040221578006799', '8.042729240792925',
       '7.614367250629397', '7.649846263031173', '7.666234175745569',
       '7.759884604571789', '7.791583909488228', '7.829769056186604',
       '7.9571426289466585', '7.654406486127567', '7.731111957408882',
       '7.792791213405503', '7.844613316074569', '7.934089886224436',
       '7.728975840249242', '7.813043343673816', '7.654713691902718',
       '7.734461970398188', '7.783192154238925', '7.874654237803034',
       '8.020945551070671', '7.547830986271524', '7.587966406959314',
       '7.661293458326776', '7.704925720936962', '8.028133557088248',
       '7.548931548136637', '7.559850661425167', '7.627537898323465',
       '7.689201872403516', '7.7141705029480026', '7.825143189743733',
       '7.889543331973465', '7.625734472182768', '7.552612173292936',
       '7.640436648343181', '7.717278360710171', '7.82518714006605',
       '7.914619909617428', '7.994079254751035', '7.631890490116242',
       '7.538796826

In [ ]:
# CONCATENATE INPUTS
def concatenate_prep_data(input_arrays, output_arrays, 
                          save_csv=False, save_txt=False):

  inputs = np.concatenate(input_arrays)
  canyonb_inputs = np.concatenate(input_arrays)
  outputs = np.concatenate(output_arrays)
  canyonb_outputs = np.concatenate(output_arrays)

  # TRANSFORM DATE AND PRESSURE INPUTS

  # method to help transform date
  def date_to_nth_day(the_date):
    date = pd.to_datetime(the_date)
    new_year_day = pd.Timestamp(year=date.year, month=1, day=1)
    day_of_the_year = (date - new_year_day).days + 1
    return day_of_the_year

  # loop through inputs and perform transformations
  for input in inputs:
    # adjust date
    date = input[0]
    frac_year = date_to_nth_day(date) / 365
    input[0] = int(date[0:4]) + frac_year 
    # # adjust pressure
    # pres = float(input[3])
    # input[3] = ( pres/20000 ) + ( 1 / ( 1 + math.exp(-1*pres / 300) )**3 ) 

  # loop through inputs and perform transformations
  for input in canyonb_inputs:
    # adjust date
    date = input[0]
    date = date[0:4]+'-'+date[4:6]+'-'+date[6:8]
    input[0] = date
    
  # if desired save arrays as .txt files
  if save_txt or save_csv: 
    if save_txt:
      input_header = 'DATE LATITUDE LONGITUDE CTDPRS CTDTMP CTDSAL OXYGEN'
      np.savetxt(data_dir + 'compiled_tpso_input.txt', inputs, 
                 fmt='%s', header=input_header)
      output_header = 'TOT_PH'
      np.savetxt(data_dir + 'compiled_ph_output.txt', outputs, 
                 fmt='%s', header=output_header)
    else:
      input_header = 'DATE, LATITUDE, LONGITUDE, CTDPRS, CTDTMP, CTDSAL, OXYGEN'
      np.savetxt(data_dir + 'compiled_tpso_input.csv', inputs, 
                 fmt='%s', delimiter=",", header=input_header)
      output_header = 'TOT_PH'
      np.savetxt(data_dir + 'compiled_ph_output.csv', outputs, 
                 fmt='%s', delimiter=",", header=output_header)
      
  # if desired save arrays as .txt files
  if save_txt or save_csv: 
    if save_txt:
      input_header = 'DATE LATITUDE LONGITUDE CTDPRS CTDTMP CTDSAL OXYGEN'
      np.savetxt(data_dir + 'canyonb_compiled_tpso_input.txt', inputs, 
                 fmt='%s', header=input_header)
      output_header = 'TOT_PH'
      np.savetxt(data_dir + 'canyonb_compiled_ph_output.txt', outputs, 
                 fmt='%s', header=output_header)
    else:
      input_header = 'DATE, LATITUDE, LONGITUDE, CTDPRS, CTDTMP, CTDSAL, OXYGEN'
      np.savetxt(data_dir + 'canyonb_compiled_tpso_input.csv', inputs, 
                 fmt='%s', delimiter=",", header=input_header)
      output_header = 'TOT_PH'
      np.savetxt(data_dir + 'canyonb_compiled_ph_output.csv', outputs, 
                 fmt='%s', delimiter=",", header=output_header)
      
  return inputs, outputs, canyonb_inputs, canyonb_outputs

# GET INPUT OUTPUT ARRAYS

inputs, outputs, canyonb_inputs, canyonb_outputs = concatenate_prep_data(
                                        (ship_input, glider_input, wcoa_input),
                                        (ship_output, glider_output, wcoa_output),
                                        save_csv=True)

display(inputs[0])
display(outputs[0])

array(['2019.358904109589', '36.798', '-121.8462', '202.6', '8.6075',
       '33.9971', '94.2'], dtype='<U19')

'7.730970739061131'

In [ ]:
count = 0
ind = 0
bad = 0
for row in inputs:
  for input in row:
    if float(input) == -999:
      bad = ind
      count += 1
  ind += 1
display(bad)
display(ind)
display(count)

inputs = np.delete(inputs, bad, 0)
outputs = np.delete(outputs, bad, 0)
canyonb_inputs = np.delete(canyonb_inputs, bad, 0)
canyonb_outputs = np.delete(canyonb_outputs, bad, 0)

count = 0
ind = 0
bad = 0
for row in inputs:
  for input in row:
    if float(input) == -999:
      bad = ind
      count += 1
  ind += 1
display(bad)
display(ind)
display(count)

102

642861

1

0

642860

0

In [ ]:
display(inputs.shape)
display(canyonb_inputs.shape)

(642860, 7)

(642860, 7)

In [ ]:
def split_input_output(inputs, outputs, test_size=0.2, val_size=0.25,
                       seed=101, shuffle=True,
                       save_txt=False, save_csv=False):
  train_inputs, test_inputs, train_outputs, test_outputs = train_test_split(
      inputs, outputs, test_size=test_size, shuffle=shuffle, random_state=seed
      )

  train_inputs, val_inputs, train_outputs, val_outputs = train_test_split(
      train_inputs, train_outputs, test_size=val_size, random_state=seed
      )
  
  # split CANYON-B
  canyonb_train_inputs, canyonb_test_inputs, canyonb_train_outputs, canyonb_test_outputs = train_test_split(
      canyonb_inputs, canyonb_outputs, 
      test_size=test_size, shuffle=shuffle, random_state=seed
      )

  canyonb_train_inputs, canyonb_val_inputs, canyonb_train_outputs, canyonb_val_outputs = train_test_split(
      canyonb_train_inputs, canyonb_train_outputs, 
      test_size=val_size, random_state=seed
      )

  if save_txt or save_csv:
    if save_txt:
      input_header = 'DATE LATITUDE LONGITUDE CTDPRS CTDTMP CTDSAL OXYGEN'
      output_header = 'TOT_PH'
      # save training sets
      np.savetxt(training_dir + 'training_tpso_input.txt', train_inputs, 
                  fmt='%s', header=input_header)
      np.savetxt(training_dir + 'training_ph_output.txt', train_outputs, 
                  fmt='%s', header=output_header)
      # save validation sets
      np.savetxt(training_dir + 'validation_tpso_input.txt', val_inputs, 
                  fmt='%s', header=input_header)
      np.savetxt(training_dir + 'validation_ph_output.txt', val_outputs, 
                  fmt='%s', header=output_header)
      # save testing sets
      np.savetxt(training_dir + 'testing_tpso_input.txt', test_inputs, 
                  fmt='%s', header=input_header)
      np.savetxt(training_dir + 'testing_ph_output.txt', test_outputs, 
                  fmt='%s', header=output_header)
    else:
      input_header = 'DATE, LATITUDE, LONGITUDE, CTDPRS, CTDTMP, CTDSAL, OXYGEN'
      output_header = 'TOT_PH'
      # save training sets
      np.savetxt(training_dir + 'training_tpso_input.csv', train_inputs, 
                  fmt='%s', delimiter=",", header=input_header)
      np.savetxt(training_dir + 'training_ph_output.csv', train_outputs, 
                  fmt='%s', delimiter=",", header=output_header)
      # save validation sets
      np.savetxt(training_dir + 'validation_tpso_input.csv', val_inputs, 
                  fmt='%s', delimiter=",", header=input_header)
      np.savetxt(training_dir + 'validation_ph_output.csv', val_outputs, 
                  fmt='%s', delimiter=",", header=output_header)
      # save testing sets
      np.savetxt(training_dir + 'testing_tpso_input.csv', test_inputs, 
                  fmt='%s', delimiter=",", header=input_header)
      np.savetxt(training_dir + 'testing_ph_output.csv', test_outputs, 
                  fmt='%s', delimiter=",", header=output_header)
      
  if save_txt or save_csv:
    if save_txt:
      input_header = 'DATE LATITUDE LONGITUDE CTDPRS CTDTMP CTDSAL OXYGEN'
      output_header = 'TOT_PH'
      # save training sets
      np.savetxt(training_dir + 'canyonb_training_tpso_input.txt', canyonb_train_inputs, 
                  fmt='%s', header=input_header)
      np.savetxt(training_dir + 'canyonb_training_ph_output.txt', canyonb_train_outputs, 
                  fmt='%s', header=output_header)
      # save validation sets
      np.savetxt(training_dir + 'canyonb_validation_tpso_input.txt', canyonb_val_inputs, 
                  fmt='%s', header=input_header)
      np.savetxt(training_dir + 'canyonb_validation_ph_output.txt', canyonb_val_outputs, 
                  fmt='%s', header=output_header)
      # save testing sets
      np.savetxt(training_dir + 'canyonb_testing_tpso_input.txt', canyonb_test_inputs, 
                  fmt='%s', header=input_header)
      np.savetxt(training_dir + 'canyonb_testing_ph_output.txt', canyonb_test_outputs, 
                  fmt='%s', header=output_header)
    else:
      input_header = 'DATE, LATITUDE, LONGITUDE, CTDPRS, CTDTMP, CTDSAL, OXYGEN'
      output_header = 'TOT_PH'
      # save training sets
      np.savetxt(training_dir + 'canyonb_training_tpso_input.csv', canyonb_train_inputs, 
                  fmt='%s', delimiter=",", header=input_header)
      np.savetxt(training_dir + 'canyonb_training_ph_output.csv', canyonb_train_outputs, 
                  fmt='%s', delimiter=",", header=output_header)
      # save validation sets
      np.savetxt(training_dir + 'canyonb_validation_tpso_input.csv', canyonb_val_inputs, 
                  fmt='%s', delimiter=",", header=input_header)
      np.savetxt(training_dir + 'canyonb_validation_ph_output.csv', canyonb_val_outputs, 
                  fmt='%s', delimiter=",", header=output_header)
      # save testing sets
      np.savetxt(training_dir + 'canyonb_testing_tpso_input.csv', canyonb_test_inputs, 
                  fmt='%s', delimiter=",", header=input_header)
      np.savetxt(training_dir + 'canyonb_testing_ph_output.csv', canyonb_test_outputs, 
                  fmt='%s', delimiter=",", header=output_header)

  return train_inputs, train_outputs, val_inputs, val_outputs, test_inputs, test_outputs


train_inputs, train_outputs, val_inputs, val_outputs, test_inputs, test_outputs = split_input_output(
    inputs, outputs, test_size=0.2, val_size=0.25, save_csv=True
)

display(train_inputs.shape)
display(train_outputs.shape)

display(val_inputs.shape)
display(val_outputs.shape)

display(test_inputs.shape)
display(test_outputs.shape)


(385716, 7)

(385716,)

(128572, 7)

(128572,)

(128572, 7)

(128572,)